In [3]:
import os
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import AutoProcessor
import glob
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from medclip.modeling_medclip import MedCLIPModel  # Make sure to use the correct MedCLIP import
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from tqdm import tqdm
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from tqdm import tqdm
from  medclip import  MedCLIPProcessor


In [4]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score, classification_report
from tqdm import tqdm
from medclip.modeling_medclip import MedCLIPModel

# === Set device to CPU ===
device = torch.device("cpu")

# === Load CSV & Encode Labels ===
csv_path = "image_dataset.csv"
df = pd.read_csv(csv_path)
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['Classification'])
n_classes = df['encoded_label'].nunique()

# === Preprocessing ===
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# === Custom Dataset ===
class MedClipDataset(Dataset):
    def __init__(self, dataframe, image_root, preprocess):
        self.dataframe = dataframe
        self.image_root = image_root
        self.preprocess = preprocess

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_root, row['Image_id'])
        label = row['encoded_label']
        image = Image.open(image_path).convert("RGB")
        image = self.preprocess(image)
        return image, torch.tensor(label, dtype=torch.long)

# === Initialize Model ===
WEIGHTS_NAME = "model_weights.pth"
model = MedCLIPModel()
model.load_state_dict(torch.load(WEIGHTS_NAME, map_location=device))
model = model.to(device)

# Attach classifier head
with torch.no_grad():
    dummy = torch.randn(1, 3, 224, 224).to(device)
    hidden_size = model.encode_image(dummy).shape[-1]
model.classifier = torch.nn.Linear(hidden_size, n_classes).to(device)

# === Dataloader ===
image_root = "BrEaST-Lesions_USG-images_and_masks"
dataset = MedClipDataset(df, image_root, preprocess)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# === Optimizer & Loss ===
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# === Training ===
epochs = 10
loss_values = []

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        images, labels = images.to(device), labels.to(device)
        feats = model.encode_image(images)
        preds = model.classifier(feats)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    loss_values.append(avg_loss)
    print(f"Epoch {epoch+1} Loss: {avg_loss:.4f}")

# === Plot Loss Curve ===
plt.figure()
plt.plot(range(1, epochs+1), loss_values, marker='o', color='blue')
plt.title("Training Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.savefig("loss_curve.png")
plt.close()

# === Evaluation ===
model.eval()
y_true, y_pred, y_scores = [], [], []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        feats = model.encode_image(images)
        preds = model.classifier(feats)
        probas = torch.softmax(preds, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(torch.argmax(probas, dim=1).cpu().numpy())
        y_scores.extend(probas.cpu().numpy())

# === Metrics ===
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="macro")
cm = confusion_matrix(y_true, y_pred)
try:
    auc = roc_auc_score(y_true, y_scores, multi_class="ovr")
except:
    auc = "AUC Error (Check classes)"

print("\n=== Evaluation Metrics ===")
print(f"Accuracy       : {acc:.4f}")
print(f"F1-Score       : {f1:.4f}")
print(f"AUC-ROC        : {auc}")
print("Confusion Matrix:")
print(cm)

# === Classification Report ===
report = classification_report(y_true, y_pred, target_names=label_encoder.classes_, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("classification_report.csv")
print("\nSaved classification_report.csv")

# === Plot Confusion Matrix ===
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig("confusion_matrix.png")
plt.close()
print("Saved confusion_matrix.png and loss_curve.png")


C:\Users\JARE WORKS\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\JARE WORKS\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1: 100%|██████████| 32/32 [01:10<00:00,  2.20s/it]


Epoch 1 Loss: 1.0941


Epoch 2: 100%|██████████| 32/32 [01:06<00:00,  2.09s/it]


Epoch 2 Loss: 1.0614


Epoch 3: 100%|██████████| 32/32 [01:19<00:00,  2.47s/it]


...

Epoch 9 Loss: 0.7566


Epoch 10: 100%|██████████| 32/32 [01:10<00:00,  2.19s/it]


Epoch 10 Loss: 0.7231


Evaluating: 100%|██████████| 32/32 [00:24<00:00,  1.29it/s]
c:\Users\JARE WORKS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\JARE WORKS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\JARE WORKS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 


=== Evaluation Metrics ===
Accuracy       : 0.9844
F1-Score       : 0.6618
AUC-ROC        : 0.6924603174603176
Confusion Matrix:
[[154   0   0]
 [  0  98   0]
 [  3   1   0]]

Saved classification_report.csv
Saved confusion_matrix.png and loss_curve.png


In [6]:
import joblib
joblib.dump(model, "results/medclip_image_model.joblib")


['results/medclip_image_model.joblib']